In [77]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

In [78]:
base_url ='https://api.fxratesapi.com'

*https://fxratesapi.com/docs/endpoints/list-available-currencies*

In [79]:
def get_currencies():
  df_currencies = pd.DataFrame()
  r = requests.get(base_url+'/currencies')
  currencies = r.json()
  for key, value in currencies.items():
    df = pd.json_normalize(value)
    df_currencies = pd.concat([df_currencies, df], ignore_index=True)
  return df_currencies

df_currencies = get_currencies()
df_currencies = df_currencies[df_currencies['code'].isin(['USD', 'ARS', 'BTC', 'ETH', 'BUSD', 'USDC', 'USDT', 'ADA', 'BNB', 'EUR', 'BRL'])].reset_index()
df_currencies

,index,code,name,decimal_digits,name_plural,rounding,symbol,symbol_native
0,5,ARS,Argentine Peso,2,Argentine pesos,0,AR$,$
1,18,BRL,Brazilian Real,2,Brazilian reals,0,R$,R$
2,42,EUR,Euro,2,Euros,0,€,€
3,100,BTC,Bitcoin,8,Bitcoins,0,₿,₿
4,101,ETH,Ethereum,18,Ethereum,0,Ξ,Ξ
5,102,BNB,Binance,8,Binance,0,BNB,BNB
6,109,ADA,Cardano,6,Cardanos,0,ADA,ADA
7,110,USDT,Tether,2,Tethers,0,USDT,USDT
8,111,USDC,USD Coin,2,USD Coins,0,USDC,USDC
9,113,BUSD,Binance USD,8,Binance USD,0,BUSD,BUSD


In [80]:
start_date = datetime(datetime.now().year, 1, 1, tzinfo=timezone.utc)
end_date = datetime.now(timezone.utc)
formato = "%Y-%m-%dT%H:%M:%S.%fZ"
start_date = start_date.strftime(formato)
end_date = end_date.strftime(formato)
print({'start_date':start_date, 'end_date':end_date})

{'start_date': '2023-01-01T00:00:00.000000Z', 'end_date': '2023-12-17T21:37:29.935771Z'}


*https://fxratesapi.com/docs/endpoints/time-series-exchange-rates*

In [81]:
def get_series_times(row):
    base_code = row['code']
    query = {
      'base': base_code,
      'start_date': start_date,
      'end_date': end_date,
#       'currencies': ['USD', 'ARS', 'BTC', 'ETH', 'BUSD', 'USDC', 'USDT', 'ADA', 'BNB', 'EUR', 'BRL'],
    }
    r = requests.get(base_url+'/timeseries', params=query)
    time_series = r.json()
#     print(time_series)
    return (
      time_series['base'] if 'base' in time_series else "N/A",
      time_series['start_date'] if 'start_date' in time_series else "N/A",
      time_series['end_date'] if 'end_date' in time_series else "N/A",
      time_series['rates'] if 'rates' in time_series else "N/A"
    )

In [82]:
df_time_series = pd.DataFrame()
df_time_series['base'], df_time_series['start_date'], df_time_series['end_date'], df_time_series['rates'] = zip(*df_currencies.apply(get_series_times, axis=1))
df_time_series

,base,start_date,end_date,rates
0,ARS,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'ARS': 1, 'ADA':..."
1,BRL,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'BRL': 1, 'ADA':..."
2,EUR,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'EUR': 1, 'ADA':..."
3,BTC,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'BTC': 1, 'ADA':..."
4,ETH,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'ETH': 1, 'ADA':..."
5,BNB,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'BNB': 1, 'ADA':..."
6,ADA,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'ADA': 1, 'AED':..."
7,USDT,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'USDT': 1, 'ADA'..."
8,USDC,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'USDC': 1, 'ADA'..."
9,BUSD,2023-01-01T00:00:00.000Z,2023-12-17T21:37:29.935Z,"{'2023-12-16T23:59:00.000Z': {'BUSD': 1, 'ADA'..."


In [83]:
df_time_series_values = pd.DataFrame()

for index, row in df_time_series.iterrows():
  base_code = row['base']
  for key, value in row['rates'].items():
#     print(value)
    df = pd.json_normalize(value)
    df['code'] = base_code
    df['date'] = key
    df_time_series_values = pd.concat([df_time_series_values, df], ignore_index=True)

df_time_series_values

,ARS,ADA,AED,AFN,ALL,AMD,ANG,AOA,ARB,AUD,...,YER,ZAR,ZMK,ZMW,ZWL,code,date,USDT,USDC,BUSD
0,1.000000,0.002048,0.004576,0.087199,0.118834,0.508098,0.002230,1.036022,0.001107,0.001860,...,0.311879,0.022791,11.218591,0.031044,7.348330,ARS,2023-12-16T23:59:00.000Z,NaN,NaN,NaN
1,1.000000,0.002064,0.004576,0.087199,0.118834,0.508098,0.002230,1.036022,0.001128,0.001860,...,0.311879,0.022791,11.218592,0.031044,7.348327,ARS,2023-12-15T23:59:00.000Z,NaN,NaN,NaN
2,1.000000,0.001948,0.004587,0.087239,0.117666,0.502767,0.002232,1.033172,0.001059,0.001864,...,0.312098,0.022887,11.242760,0.030843,7.357599,ARS,2023-12-14T23:59:00.000Z,NaN,NaN,NaN
3,1.000000,0.001885,0.004589,0.087047,0.117412,0.500376,0.002233,1.028438,0.001057,0.001875,...,0.312287,0.023283,11.250072,0.030643,7.349614,ARS,2023-12-13T23:59:00.000Z,NaN,NaN,NaN
4,1.000000,0.004736,0.010020,0.189808,0.257506,1.100826,0.004874,2.259783,0.002176,0.004157,...,0.681693,0.051677,24.566168,0.066840,15.878391,ARS,2023-12-12T23:59:00.000Z,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,178.909824,3.705081,3.673005,88.500164,111.320181,394.780494,1.803183,503.685928,NaN,1.480299,...,250.300476,17.165934,9001.200000,18.214211,321.999592,USD,2023-01-05T23:59:00.000Z,NaN,NaN,NaN
3428,178.571917,3.743071,3.673007,87.500089,107.150211,394.490559,1.802767,503.685925,NaN,1.464303,...,250.300497,16.885419,9001.200000,18.185184,321.999592,USD,2023-01-04T23:59:00.000Z,NaN,NaN,NaN
3429,178.380409,3.941406,3.673004,87.500167,107.150123,394.690441,1.802417,503.609827,NaN,1.486770,...,250.300451,17.027022,9001.200000,18.109730,321.999592,USD,2023-01-03T23:59:00.000Z,NaN,NaN,NaN
3430,178.152844,3.940376,3.672705,89.221443,106.959021,393.605864,1.802400,503.633924,NaN,1.469446,...,250.300265,17.006971,9001.200000,18.076790,321.999592,USD,2023-01-02T23:59:00.000Z,NaN,NaN,NaN


In [87]:
df_final = df_currencies.merge(df_time_series_values, on='code')
df_final.to_csv('../CSV/df_time_series_2023.csv')
df_final[df_final.code == 'USD'].to_csv('../CSV/df_time_series_USD_2023.csv')
df_final[(df_final.date.str.contains('2023-12'))&(df_final.code == 'USD')].to_csv('../CSV/df_time_series_USD_2023-12.csv')
df_final

,index,code,name,decimal_digits,name_plural,rounding,symbol,symbol_native,ARS,ADA,...,XRP,YER,ZAR,ZMK,ZMW,ZWL,date,USDT,USDC,BUSD
0,5,ARS,Argentine Peso,2,Argentine pesos,0,AR$,$,1.000000,0.002048,...,0.001994,0.311879,0.022791,11.218591,0.031044,7.348330,2023-12-16T23:59:00.000Z,NaN,NaN,NaN
1,5,ARS,Argentine Peso,2,Argentine pesos,0,AR$,$,1.000000,0.002064,...,0.002008,0.311879,0.022791,11.218592,0.031044,7.348327,2023-12-15T23:59:00.000Z,NaN,NaN,NaN
2,5,ARS,Argentine Peso,2,Argentine pesos,0,AR$,$,1.000000,0.001948,...,0.001974,0.312098,0.022887,11.242760,0.030843,7.357599,2023-12-14T23:59:00.000Z,NaN,NaN,NaN
3,5,ARS,Argentine Peso,2,Argentine pesos,0,AR$,$,1.000000,0.001885,...,0.001986,0.312287,0.023283,11.250072,0.030643,7.349614,2023-12-13T23:59:00.000Z,NaN,NaN,NaN
4,5,ARS,Argentine Peso,2,Argentine pesos,0,AR$,$,1.000000,0.004736,...,0.004377,0.681693,0.051677,24.566168,0.066840,15.878391,2023-12-12T23:59:00.000Z,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,171,USD,US Dollar,2,US dollars,0,$,$,178.909824,3.705081,...,2.944619,250.300476,17.165934,9001.200000,18.214211,321.999592,2023-01-05T23:59:00.000Z,NaN,NaN,NaN
3428,171,USD,US Dollar,2,US dollars,0,$,$,178.571917,3.743071,...,2.880227,250.300497,16.885419,9001.200000,18.185184,321.999592,2023-01-04T23:59:00.000Z,NaN,NaN,NaN
3429,171,USD,US Dollar,2,US dollars,0,$,$,178.380409,3.941406,...,2.894068,250.300451,17.027022,9001.200000,18.109730,321.999592,2023-01-03T23:59:00.000Z,NaN,NaN,NaN
3430,171,USD,US Dollar,2,US dollars,0,$,$,178.152844,3.940376,...,2.860643,250.300265,17.006971,9001.200000,18.076790,321.999592,2023-01-02T23:59:00.000Z,NaN,NaN,NaN
